# NIV-DARPA Experiment 05: Debt Efficiency Validation
Computes the Debt-Efficiency Index (DEI) using cumulative regeneration and friction flows from Treasury data.

In [ ]:
from niv.data.treasury_data import TreasuryData, NIVMetrics, get_logger
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

logger = get_logger('Exp05')
Path('visuals').mkdir(exist_ok=True)

td = TreasuryData(cache_dir='data/fred_cache')
df = td.fetch('2010-01-01')
df = td.normalize(df)

# Compute DEI
df['FrictionCum'] = df['F_t'].cumsum()
df['RegenerationCum'] = df['P_t'].cumsum()
df['DEI'] = df['RegenerationCum'] / (df['FrictionCum'] + 1e-9)

# Compute NIV series
metrics = NIVMetrics()
df['NIV_index'] = metrics.compute_niv_series(df)

fig, ax = plt.subplots(2,1,figsize=(9,6), sharex=True)
ax[0].plot(df.index, df['NIV_index'], label='NIV Index', color='steelblue')
ax[0].set_ylabel('NIV')
ax[0].legend()

ax[1].plot(df.index, df['DEI'], label='Debt-Efficiency Index', color='seagreen')
ax[1].set_ylabel('DEI')
ax[1].legend()

plt.suptitle('U.S. Treasury NIV & Debt-Efficiency Index (2010–2024)')
plt.tight_layout()
plt.savefig('visuals/exp05_debt_efficiency_validation.png', dpi=250)
plt.show()

logger.info('Debt-Efficiency validation complete.')
logger.info(f'Average DEI: {df["DEI"].mean():.4f}')